# Baseline model

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

path = 'mypath'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
xr_train = xr.open_dataset(path+'xr_train_timestamped.netcdf')
xr_test = xr.open_dataset(path+'xr_test_timestamped.netcdf')

In [ ]:
from torch.utils.data import Dataset


def get_seq_length(past_dict = {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': 1},
                   future_dict = {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': 1},
                   max_len = 133992):
  look_up = (past_dict['hour'] * 4 + past_dict['day'] * 24 * 4
            + past_dict['month'] *30 * 24 * 4 + past_dict['year'] * 12 * 30 * 24 * 4)
  look_ahead = (future_dict['hour'] * 4 + future_dict['day'] * 24 * 4
                + future_dict['month'] *30 * 24 * 4 + future_dict['year'] * 12 * 30 * 24 * 4)

  if look_up+look_ahead > max_len:
    print('Not enough data to accomodate requested sequence lengths, returning 1 and 1')
    look_up, look_ahead = 1, 1

  return look_up, look_ahead


class PVDataset(Dataset):
  def __init__(self, X, look_up=1, look_ahead=1):

    '''
    X = xarray dataset
    look_up = int, length of sequence of past observations to use for prediction
    look_ahead = int, length of sequence to predict
    '''

    self.X = X
    self.stations = list(X.keys())
    self.look_up = look_up
    self.look_ahead = look_ahead
    self.shape_0 = X.dims['datetime']
    self.shape_1 = len(self.stations)
    self.data_len = self.shape_0 // (self.look_up + self.look_ahead)

  def __len__(self):
    return self.data_len * self.shape_1

  def __getitem__(self, i):

    ss_num = i // self.data_len
    ss_id = self.stations[ss_num]
    i = (i % self.shape_1) * (self.look_up + self.look_ahead)

    t = i + self.look_up

    while sum(self.X[ss_id].data[t-self.look_up : t]) < 0.5:
      i = np.random.choice(list(range(self.__len__())), 1)[0]
      ss_num = i // self.data_len
      ss_id = self.stations[ss_num]
      i = (i % self.shape_1) * (self.look_up + self.look_ahead)

      t = i + self.look_up

    x_item = np.zeros((self.look_up, 5))
    y_item = np.zeros((self.look_ahead, 1))

    x_item[:, 0], y_item = self.X[ss_id].data[t-self.look_up : t], self.X[ss_id].data[t : t+self.look_ahead]
    x_item[:, 1] = self.X.date_sin.data[t-self.look_up : t]
    x_item[:, 2] = self.X.date_cos.data[t-self.look_up : t]
    x_item[:, 3] = self.X.time_sin.data[t-self.look_up : t]
    x_item[:, 4] = self.X.time_sin.data[t-self.look_up : t]

    return torch.Tensor(x_item).float(), torch.Tensor(y_item).float()

In [ ]:
from torch.utils.data import DataLoader

def loaders(xr_test, xr_train, batch_size, dict_look_up, dict_look_ahead, sample=False):

  max_len = len(xr_train['datetime'].data)

  look_up, look_ahead = get_seq_length(past_dict=dict_look_up, future_dict=dict_look_ahead, max_len=max_len)

  train_dataset = PVDataset(xr_train, look_up, look_ahead)
  test_dataset = PVDataset(xr_test, look_up, look_ahead)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

  print("Expected number of steps is %d" %((train_dataset.__len__()  + batch_size - 1) // batch_size))

  return train_loader, test_loader, look_ahead

In [ ]:
class LSTM_model(nn.Module):

  def __init__(self, hidden_size,
               num_layers,
               out_features,
               input_size,
               dropout=0):

    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        batch_first=True,
                        dropout=dropout
                        )

    self.fc = nn.Linear(in_features=hidden_size, out_features=out_features)

  def forward(self, x):

    batch_size = x.size(0)

    h_t = torch.zeros(self.num_layers,
                      batch_size,
                      self.hidden_size).to(device)
    c_t = torch.zeros(self.num_layers,
                      batch_size,
                      self.hidden_size).to(device)


    _, (h_out, c_n) = self.lstm(x, (h_t, c_t))

    h_out = h_out[0].view(-1, self.hidden_size)

    out = self.fc(h_out)

    return out

In [ ]:
def train_epoch(model, train_loader, optimizer, loss_function, print_every=500):
  loss_array = []
  model.train(True)
  running_loss = 0.

  for batch_index, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    output = model(x_batch)

    loss = loss_function(output, y_batch.squeeze())
    running_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_index % print_every == 0:
      loss_array.append(running_loss/print_every)
      print("Batch %d, loss: %f" %(batch_index+1, running_loss/print_every))
      running_loss=0.

  loss_array.append(running_loss/((batch_index+1)%1000))

  print()
  return loss_array

In [ ]:
def validate_epoch(model, test_loader, loss_function):
  model.train(False)
  running_loss = 0.

  for batch_index, batch in enumerate(test_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    with torch.no_grad():
      output = model(x_batch)
      loss = loss_function(output, y_batch.squeeze())
      running_loss += loss.item()

  avg_loss = running_loss / len(test_loader)

  print('Val loss: {0:.5f}'.format(avg_loss))
  print('_______________________________')
  print()

  return avg_loss

### best model

In [ ]:
best_params_df = pd.read_csv(path+'baseline_best_params.csv')
best_params = best_params_df.loc[0].to_dict()

In [ ]:
data_params = {
    'xr_test':  xr_test,
    'xr_train': xr_train,
    'batch_size': 64,
    'dict_look_up': {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': int(best_params['look_up'])},
    'dict_look_ahead': {'year': 0, 'month': 0, 'week': 0, 'day': 0, 'hour': 6},
}

train_loader, test_loader, look_ahead = loaders(**data_params)

model_params = {
    'input_size': 5,
    'hidden_size':  int(best_params['hidden_size']),
    'num_layers':   int(best_params['num_layers']),
    'out_features': look_ahead,
    'dropout':      best_params['dropout']
}

print(model_params)

model = LSTM_model(**model_params)

model.to(device)

Expected number of steps is 3052
{'input_size': 5, 'hidden_size': 64, 'num_layers': 3, 'out_features': 24, 'dropout': 0.113318450480939}


LSTM_model(
  (lstm): LSTM(5, 64, num_layers=3, batch_first=True, dropout=0.113318450480939)
  (fc): Linear(in_features=64, out_features=24, bias=True)
)

In [ ]:
learning_rate = 0.001
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 6

train_loss = []
test_loss = []

best_loss = 1
best_epoch = 0
for epoch in range(num_epochs):
  print('Epoch:', epoch+1)
  train_loss.append(train_epoch(model=model, train_loader=train_loader,
                                optimizer=optimizer,
                                loss_function=loss_function,
                                print_every=300))
  test_loss.append(validate_epoch(model=model, test_loader=test_loader, loss_function=loss_function))
  if test_loss[-1] < best_loss:
    best_loss = test_loss[-1]
    best_epoch = epoch
  if epoch - best_epoch >= 2:
    break

In [ ]:
torch.save(model.state_dict(), path+'baseline_model')